In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.5 MB/s eta 0:00:00


In [4]:
!pip install torchvision 
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# !pip install bert-extractive-summarizer

In [6]:
# from summarizer import Summarizer,TransformerSummarizer

In [7]:
# from transformers import DistilBertModel, DistilBertTokenizer
# distilbert_model = DistilBertModel.from_pretrained('distilbert-base-uncased', output_hidden_states=True)
# distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [8]:
# distilbert_model = Summarizer(custom_model=distilbert_model, custom_tokenizer=distilbert_tokenizer, random_state = 7)

In [9]:
# print(distilbert_model)

In [10]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ccdv/lsg-bart-large-4096")

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [12]:

model = AutoModel.from_pretrained("ccdv/lsg-bart-large-4096", trust_remote_code=True).to(device)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Some weights of the model checkpoint at ccdv/lsg-bart-large-4096 were not used when initializing LSGBartModel: ['final_logits_bias', 'lm_head.weight']
- This IS expected if you are initializing LSGBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LSGBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
from transformers import AutoModel

model = AutoModel.from_pretrained("ccdv/lsg-bart-large-4096", 
    trust_remote_code=True, 
    num_global_tokens=16,
    block_size=64,
    sparse_block_size=64,
    sparsity_factor=4,
    sparsity_type="none",
    mask_first_token=True
)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Some weights of the model checkpoint at ccdv/lsg-bart-large-4096 were not used when initializing LSGBartModel: ['final_logits_bias', 'lm_head.weight']
- This IS expected if you are initializing LSGBartModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LSGBartModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
# from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [15]:
max_input = 512
max_target = 128
batch_size = 1
model_checkpoints = "ccdv/lsg-bart-large-4096"

In [16]:
!pip install datasets
import datasets
from datasets import load_dataset, load_metric, load_from_disk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.9/140.9 KB 15.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [17]:
import pandas as pd

In [18]:
df1 = pd.read_csv('/content/drive/MyDrive/NLP Final/bert_passed_train.csv')
train_dataset = datasets.Dataset.from_pandas(df1)

In [19]:
df2 = pd.read_csv('/content/drive/MyDrive/NLP Final/bert_passed_validation.csv')
validation_dataset = datasets.Dataset.from_pandas(df2)

In [20]:
padding = "max_length"
ignore_pad_token_for_loss = True

In [21]:
train_dataset

Dataset({
    features: ['Filename', 'judgement', 'summary'],
    num_rows: 799
})

In [22]:
validation_dataset

Dataset({
    features: ['Filename', 'judgement', 'summary'],
    num_rows: 100
})

In [23]:
def preprocess_data(data_to_process):
  #get the dialogue text
  inputs = [dialogue for dialogue in data_to_process['judgement']]
  #tokenize text
  model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

  #tokenize labels
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data_to_process['summary'], max_length=max_target, padding='max_length', truncation=True)
    
  model_inputs['labels'] = targets['input_ids']
  #reuturns input_ids, attention_masks, labels
  return model_inputs

In [24]:
train_tokenize_data = train_dataset.map(preprocess_data, batched = True, remove_columns=['Filename', 'judgement', 'summary'])


Map:   0%|          | 0/799 [00:00<?, ? examples/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3586: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [25]:
train_tokenize_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 799
})

In [26]:
validation_tokenize_data = validation_dataset.map(preprocess_data, batched = True, remove_columns=['Filename', 'judgement', 'summary'])


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [27]:
validation_tokenize_data

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [28]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [29]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints, trust_remote_code = True)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


In [30]:
model.config

LSGBartConfig {
  "_name_or_path": "ccdv/lsg-bart-large-4096",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "adaptive": true,
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "LSGBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "auto_map": {
    "AutoModel": "modeling_lsg_bart.LSGBartModel",
    "AutoModelForCausalLM": "modeling_lsg_bart.LSGBartForCausalLM",
    "AutoModelForQuestionAnswering": "modeling_lsg_bart.LSGBartForQuestionAnswering",
    "AutoModelForSeq2SeqLM": "modeling_lsg_bart.LSGBartForConditionalGeneration",
    "AutoModelForSequenceClassification": "modeling_lsg_bart.LSGBartForSequenceClassification"
  },
  "base_model_prefix": "lsg",
  "block_size": 128,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout

In [31]:
batch_size = 2

In [32]:
label_pad_token_id = -100

In [33]:
model.config

LSGBartConfig {
  "_name_or_path": "ccdv/lsg-bart-large-4096",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "adaptive": true,
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "LSGBartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "auto_map": {
    "AutoModel": "modeling_lsg_bart.LSGBartModel",
    "AutoModelForCausalLM": "modeling_lsg_bart.LSGBartForCausalLM",
    "AutoModelForQuestionAnswering": "modeling_lsg_bart.LSGBartForQuestionAnswering",
    "AutoModelForSeq2SeqLM": "modeling_lsg_bart.LSGBartForConditionalGeneration",
    "AutoModelForSequenceClassification": "modeling_lsg_bart.LSGBartForSequenceClassification"
  },
  "base_model_prefix": "lsg",
  "block_size": 128,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout

In [34]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=label_pad_token_id,)

In [35]:
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value.mid.fmeasure * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [36]:
args = transformers.Seq2SeqTrainingArguments(
    'conversation-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=5,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=False
    )

In [37]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=d9887957bd9688c2a718e16a2f116930affc04e94ca990ac1c8f620d2f440f8a
  Stored in directory: /root/.cache/pip/wheels/24/55/6f/ebfc4cb176d1c9665da4e306e1705496206d08215c1acd9dde
Successfully built rouge_score


In [38]:
metric = load_metric('rouge')

<ipython-input-38-10e4a614ebd6>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('rouge')


In [39]:
!pip install torchvision 
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
trainer = transformers.Seq2SeqTrainer(
    model, 
    args,
    train_dataset=train_tokenize_data,
    eval_dataset=validation_tokenize_data,
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [41]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,No log,2.827291,12.453000,4.361800,10.613600,11.171000,20.000000
2,2.771300,2.749600,11.917600,4.773500,10.466100,10.919400,20.000000
2,2.239500,2.707231,12.034200,4.173200,10.242100,10.799800,20.000000
4,1.978700,2.703847,12.551500,4.429800,10.734700,11.130500,20.000000
4,1.978700,2.708022,12.303100,4.352400,10.399700,10.978600,20.000000


TrainOutput(global_step=1995, training_loss=2.1944780163298874, metrics={'train_runtime': 1968.3079, 'train_samples_per_second': 2.03, 'train_steps_per_second': 1.014, 'total_flos': 4323373681213440.0, 'train_loss': 2.1944780163298874, 'epoch': 4.99})

## REFERENCES

Inspired from: 

https://github.com/AldoF95/bart-chat-summarizer-finetuning/blob/main/Bart_large_xsum_fine_tuned_samsum.ipynb

https://huggingface.co/docs/transformers/model_doc/bart

https://huggingface.co/docs/transformers/model_doc/bert